Predict script for Split model, predict the D and R matrices, and visualize the result.

In [1]:
import os
from dataset.dataset import ImageDataset
from modules.split_modules import SplitModel
import json
from PIL import Image
import torch
from torchsummary import summary
import numpy as np
import cv2
import json

In [2]:
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [9]:
# load dataset
folder = 'train'
with open('D:/dataset/table/table_line/Split1/'+ folder+'_labels.json', 'r') as f:
    labels = json.load(f)
dataset = ImageDataset('D:/dataset/table/table_line/Split1/'+ folder+'_input', labels, 8, scale=0.25)

In [4]:
# init model
net = SplitModel(3)
net = torch.nn.DataParallel(net).cuda()

In [5]:
# load saved checkpoint
net.load_state_dict(torch.load('split_model.pth'))

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

In [6]:
# change to eval mode
net.eval()

DataParallel(
  (module): SplitModel(
    (sfcn): SFCN(
      (conv1): Sequential(
        (0): Conv2d(3, 18, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), bias=False)
        (1): ReLU(inplace)
      )
      (conv2): Sequential(
        (0): Conv2d(18, 18, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), bias=False)
        (1): ReLU(inplace)
      )
      (conv3): Sequential(
        (0): Conv2d(18, 18, kernel_size=(7, 7), stride=(1, 1), padding=(6, 6), dilation=(2, 2), bias=False)
        (1): ReLU(inplace)
      )
    )
    (rpn1): ProjectionNet(
      (conv_branch1): Sequential(
        (0): Conv2d(18, 6, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2), dilation=(2, 2))
        (1): GroupNorm(3, 6, eps=1e-05, affine=True)
        (2): ReLU(inplace)
      )
      (conv_branch2): Sequential(
        (0): Conv2d(18, 6, kernel_size=(3, 3), stride=(1, 1), padding=(3, 3), dilation=(3, 3))
        (1): GroupNorm(3, 6, eps=1e-05, affine=True)
        (2): ReLU(inplace)
      )
  

In [24]:
img, label = dataset[50]
r,c = net(img.unsqueeze(0))
r = r[-1]>0.5
c = c[-1]>0.5
c = c.cpu().detach().numpy()
r = r.cpu().detach().numpy()
r_im = r.reshape((-1,1))*np.ones((r.shape[0],c.shape[0]))
c_im = c.reshape((1,-1))*np.ones((r.shape[0],c.shape[0]))
im = cv2.bitwise_or(r_im,c_im)

In [25]:
Image.fromarray(img.numpy()[2]*255.).convert('L')

In [26]:
Image.fromarray(im*255.).convert('L')